Reformat "500 Cities" to make it more useable.

More intuiitive way of organizing data-frame would be
- each of the 28,000 census-tracts as an individual row
- separate column for each health measurement within the row

Requires a few steps:
1. Change dataframe format
2. Pulling latitude and longitude using regex
3. creating 5 digit FIPS code from unique ID
4. Combining "500 Cities" with income data


In [1]:
import pandas as pd

In [2]:
raw_df = pd.read_csv("/Users/abeygailpanganiban/Desktop/practice/500_Cities__Local_Data_for_Better_Health__2019_release.csv.crdownload")

In [33]:
raw_df.sample(5)

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
478234,2016,NJ,New Jersey,Trenton,Census Tract,BRFSS,Prevention,3474000-34021001300,Older adult women aged >=65 Years who are up t...,%,...,32.8,NaN,NaN,4216,"(40.2408242792, -74.794302758)",PREVENT,COREW,3474000.0,3.402100e+10,Core preventive services for older women
296668,2016,GA,Georgia,Columbus,Census Tract,BRFSS,Prevention,1319000-13215000800,"Fecal occult blood test, sigmoidoscopy, or col...",%,...,61.7,NaN,NaN,2431,"(32.4983768258, -84.9706380063)",PREVENT,COLON_SCREEN,1319000.0,1.321500e+10,Colorectal Cancer Screening
736099,2017,TX,Texas,McAllen,Census Tract,BRFSS,Health Outcomes,4845384-48215020503,Physical health not good for >=14 days among a...,%,...,17.8,NaN,NaN,8586,"(26.1599321818, -98.266675704)",HLTHOUT,PHLTH,4845384.0,4.821502e+10,Physical Health
773655,2017,VA,Virginia,Hampton,Census Tract,BRFSS,Health Outcomes,5135000-51650010307,Coronary heart disease among adults aged >=18 ...,%,...,5.8,NaN,NaN,5339,"(37.0579277371, -76.4395661024)",HLTHOUT,CHD,5135000.0,5.165001e+10,Coronary Heart Disease
516597,2016,NY,New York,New York,Census Tract,BRFSS,Prevention,3651000-36081025100,Older adult men aged >=65 Years who are up to ...,%,...,33.5,NaN,NaN,5720,"(40.7449234091, -73.9096532213)",PREVENT,COREM,3651000.0,3.608103e+10,Core preventive services for older men


In [3]:
raw_df.shape

(810103, 24)

In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810103 entries, 0 to 810102
Data columns (total 24 columns):
Year                          810103 non-null int64
StateAbbr                     810103 non-null object
StateDesc                     810103 non-null object
CityName                      810047 non-null object
GeographicLevel               810103 non-null object
DataSource                    810103 non-null object
Category                      810103 non-null object
UniqueID                      810103 non-null object
Measure                       810103 non-null object
Data_Value_Unit               810103 non-null object
DataValueTypeID               810103 non-null object
Data_Value_Type               810103 non-null object
Data_Value                    787311 non-null float64
Low_Confidence_Limit          787311 non-null float64
High_Confidence_Limit         787311 non-null float64
Data_Value_Footnote_Symbol    22794 non-null object
Data_Value_Footnote           22794 non-

In [5]:
list(raw_df['GeographicLevel'].unique())

['Census Tract', 'City', 'US']

In [8]:
#Mask data-frame to isolate either city-level data or census-tract-level data

census_df = raw_df[raw_df["GeographicLevel"] == "Census Tract"]

In [26]:
#Look at unique values in "Measure" column

measure_dict = dict(census_df['Measure'].value_counts())
measures_of_interest = list(measure_dict.keys())
print(measures_of_interest)

['Visits to dentist or dental clinic among adults aged >=18 Years', 'Chronic kidney disease among adults aged >=18 Years', 'Sleeping less than 7 hours among adults aged >=18 Years', 'No leisure-time physical activity among adults aged >=18 Years', 'Papanicolaou smear use among adult women aged 21\xe2\x80\x9365 Years', 'Current asthma among adults aged >=18 Years', 'Arthritis among adults aged >=18 Years', 'High blood pressure among adults aged >=18 Years', 'Cholesterol screening among adults aged >=18 Years', 'Cancer (excluding skin cancer) among adults aged >=18 Years', 'Older adult women aged >=65 Years who are up to date on a core set of clinical preventive services: Flu shot past Year, PPV shot ever, Colorectal cancer screening, and Mammogram past 2 Years', 'Visits to doctor for routine checkup within the past Year among adults aged >=18 Years', 'Current lack of health insurance among adults aged 18\xe2\x80\x9364 Years', 'Physical health not good for >=14 days among adults aged >=1

In [28]:
census_df['Measure'].head(5)

0              Arthritis among adults aged >=18 Years
5              Arthritis among adults aged >=18 Years
8     Diagnosed diabetes among adults aged >=18 Years
15       Mammography use among women aged 50–74 Years
18    Diagnosed diabetes among adults aged >=18 Years
Name: Measure, dtype: object

If you look at the columns of census_data, you will see a column labeled "Measure"

Look at the unique values showing all the health measures that were collected. 

Need to create a list of any and all health-measures of interest, ensuring they are spelled exactly how they appear

In [37]:
data_frame_list = [census_df[census_df['Measure'] == x] for x in measures_of_interest]
for df in data_frame_list:
    print type(df)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pand